In [1]:
print("hello")

hello


In [2]:
import os
os.chdir('../')

In [3]:
from src.utils.minio_uploader import MinioUploader
from src.utils.delta_exporter import *
from src.components.data_ingestion import *
import logging
import csv
from datetime import datetime, timedelta, timezone
from binance_sdk_c2c.c2c import C2C, ConfigurationRestAPI, C2C_REST_API_PROD_URL
from turtle import home
from typing import List, Optional
from binance_sdk_c2c.rest_api.models import GetC2CTradeHistoryResponse, GetC2CTradeHistoryResponseDataInner
from src.utils.utils import *
from src.components.data_ingestion import C2CExtended

# Test

In [4]:
from binance_common.configuration import ConfigurationRestAPI
# Create configuration for the REST API
configuration_rest_api = ConfigurationRestAPI(
    api_key=os.getenv("API_KEY", ""),
    api_secret=os.getenv("API_SECRET", ""),
    base_path=os.getenv("BASE_PATH", C2C_REST_API_PROD_URL),
)
c2c = C2CExtended(configuration_rest_api)
# Ví dụ 5: Lấy giao dịch khoảng thời gian tùy chỉnh (01/09/2025 - 10/09/2025)
# Lấy giao dịch cho danh sách các ngày bất kỳ
data = c2c.get_prev_week_data()

INFO:root:Page 1 rate limits: []
INFO:root:Page 1 retrieved 50 records, 0 within time range
INFO:root:Consecutive empty pages: 1/3
INFO:root:Page 2 rate limits: []
INFO:root:Page 2 retrieved 50 records, 35 within time range
INFO:root:Page 3 rate limits: []
INFO:root:Page 3 retrieved 50 records, 50 within time range
INFO:root:Page 4 rate limits: []
INFO:root:Page 4 retrieved 50 records, 50 within time range
INFO:root:Page 5 rate limits: []
INFO:root:Page 5 retrieved 50 records, 50 within time range
INFO:root:Page 6 rate limits: []
INFO:root:Page 6 retrieved 50 records, 50 within time range
INFO:root:Page 7 rate limits: []
INFO:root:Page 7 retrieved 50 records, 50 within time range
INFO:root:Page 8 rate limits: []
INFO:root:Page 8 retrieved 50 records, 50 within time range
INFO:root:Page 9 rate limits: []
INFO:root:Page 9 retrieved 50 records, 50 within time range
INFO:root:Page 10 rate limits: []
INFO:root:Page 10 retrieved 50 records, 50 within time range
INFO:root:Page 11 rate limits:

## Convert API's response data to deltalake

In [5]:
len(data)

943

In [6]:
output_file = write_to_csv(data, "thisweek")

INFO:root:Trade history written to c2c_trade_history_thisweek.csv with 943 records


In [9]:
# 1) Convert API results (list[dict] or objects) to DataFrame
df = api_results_to_dataframe(data)


2025-10-25 12:33:43.308 | INFO     | src.utils.delta_exporter:api_results_to_dataframe:50 - Created DataFrame: shape=(943, 18)


In [10]:
df.shape

(943, 18)

In [11]:
delta_path = "c2c"

In [12]:
write_dataframe_to_delta(df, delta_path, mode="append", partition_by=None)

2025-10-25 12:33:47.877 | INFO     | src.utils.delta_exporter:write_dataframe_to_delta:85 - Wrote DataFrame to Delta: path=c2c, mode=append, rows=943


'c2c'

In [13]:
test_bucket = "bronze"

In [11]:
uploader = MinioUploader(endpoint = "localhost:9000", access_key = "adminc2c", secret_key = "adminc2c", secure = False, region = None)

In [12]:
upload_delta_to_minio(delta_path = delta_path, uploader = uploader, bucket = test_bucket, dest_prefix = "c2c_trades")

2025-10-21 11:32:46.325 | INFO     | src.utils.minio_uploader:ensure_bucket:30 - Created bucket: bronze


2025-10-21 11:32:46.409 | INFO     | src.utils.minio_uploader:upload_file:57 - Uploaded file d:\Working\binance-merchant-trading-flow\c2c\part-00001-c40d3cd0-d35d-474f-af42-0a047b84331a-c000.snappy.parquet -> s3://bronze/c2c_trades/part-00001-c40d3cd0-d35d-474f-af42-0a047b84331a-c000.snappy.parquet
2025-10-21 11:32:46.471 | INFO     | src.utils.minio_uploader:upload_file:57 - Uploaded file d:\Working\binance-merchant-trading-flow\c2c\_delta_log\00000000000000000000.json -> s3://bronze/c2c_trades/_delta_log/00000000000000000000.json
2025-10-21 11:32:46.474 | INFO     | src.utils.delta_exporter:upload_delta_to_minio:109 - Uploaded Delta folder to s3://bronze/c2c_trades


# Spark Transformation - Đọc và xử lý Parquet files

In [7]:
from src.utils.spark_session import *


In [22]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("P2P_Binance_Bronze_to_Silver")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")

print("✅ SparkSession initialized")

# Set log level to reduce noise
spark.sparkContext.setLogLevel("WARN")

print("SparkSession initialized successfully!")
print(f"Spark version: {spark.version}")

✅ SparkSession initialized
SparkSession initialized successfully!
Spark version: 3.5.3


In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_unixtime, to_date

spark = (
    SparkSession.builder
    .appName("C2C Partition by Date")
    # Thêm Delta Lake extension
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    # Thêm gói Delta Lake (chú ý version khớp Spark)
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
    .getOrCreate()
)


In [23]:
# Đọc file Parquet từ Bronze zone
# Giả sử schema đã infer từ Parquet (nếu không, thêm .option("inferSchema", "true"))
df_bronze = spark.read.parquet("./c2c/data.parquet")

# Hiển thị schema và sample data để kiểm tra
df_bronze.printSchema()
df_bronze.show(5, truncate=False)

# Đếm tổng số records
print(f"Total records in Bronze: {df_bronze.count()}")

root
 |-- orderNumber: string (nullable = true)
 |-- advNo: string (nullable = true)
 |-- tradeType: string (nullable = true)
 |-- asset: string (nullable = true)
 |-- fiat: string (nullable = true)
 |-- fiatSymbol: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- totalPrice: string (nullable = true)
 |-- unitPrice: string (nullable = true)
 |-- orderStatus: string (nullable = true)
 |-- createTime: long (nullable = true)
 |-- commission: string (nullable = true)
 |-- counterPartNickName: string (nullable = true)
 |-- takerCommissionRate: string (nullable = true)
 |-- takerCommission: string (nullable = true)
 |-- takerAmount: string (nullable = true)
 |-- payMethodName: string (nullable = true)
 |-- additionalKycVerify: long (nullable = true)



Py4JJavaError: An error occurred while calling o73.showString.
: org.apache.spark.SparkException: Cannot find catalog plugin class for catalog 'spark_catalog': org.apache.spark.sql.delta.catalog.DeltaCatalog.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.catalogPluginClassNotFoundForCatalogError(QueryExecutionErrors.scala:1926)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:70)
	at org.apache.spark.sql.connector.catalog.CatalogManager.loadV2SessionCatalog(CatalogManager.scala:68)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$v2SessionCatalog$2(CatalogManager.scala:87)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$v2SessionCatalog$1(CatalogManager.scala:87)
	at scala.Option.map(Option.scala:230)
	at org.apache.spark.sql.connector.catalog.CatalogManager.v2SessionCatalog(CatalogManager.scala:86)
	at org.apache.spark.sql.connector.catal

In [17]:
from pyspark.sql.functions import col, from_unixtime, to_date
df_transformed = (
    df_bronze
    .withColumn("createTime_ts", (from_unixtime((col("createTime") / 1000) - 7 * 3600)))  # timestamp chuẩn UTC
    .withColumn("trade_date", to_date(col("createTime_ts")))  # chỉ lấy ngày
)

In [18]:
df_transformed.select("createTime", "createTime_ts", "trade_date").show(5, truncate=False)

+-------------+-------------------+----------+
|createTime   |createTime_ts      |trade_date|
+-------------+-------------------+----------+
|1760338117635|2025-10-13 06:48:37|2025-10-13|
|1760338134989|2025-10-13 06:48:54|2025-10-13|
|1760338139930|2025-10-13 06:48:59|2025-10-13|
|1760338147813|2025-10-13 06:49:07|2025-10-13|
|1760338148494|2025-10-13 06:49:08|2025-10-13|
+-------------+-------------------+----------+
only showing top 5 rows



In [19]:
(
    df_transformed
    .write
    .format("delta")
    .mode("overwrite")  # hoặc "append" nếu muốn incremental load
    .partitionBy("trade_date")
    .save("./c2c_deltalake/")  # đường dẫn Silver/Gold zone
)


Py4JJavaError: An error occurred while calling o56.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: delta. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:725)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:873)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:260)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:243)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/